In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from GPy.models import GPRegression
from GPy.kern import Matern52, Exponential
from summit.utils.models import GPyModel, ModelGroup
from summit.utils.dataset import DataSet
from GPy.inference.optimization import Adam, RProp, Optimizer
from scipydirect import minimize as direct
import numpy as np

In [3]:
ds = DataSet.read_csv(f'data/python/20200604/experiment_0.csv')

## LBSG Optimization Algorithm

In [15]:
models = ModelGroup({f'y_{i}': GPyModel(input_dim = 5) for i in range(2)})
lhs_data = ds[ds['strategy']=="LHS"]
inputs = [f'x_{i}' for i in range(5)]
models.fit(lhs_data[inputs], lhs_data[['y_0', 'y_1']], num_restarts=100, parallel=True)

In [17]:
print("Log Likelihood:", models['y_1']._model.log_likelihood())
models['y_1']._model

Log Likelihood: -8.746008630737428


GP_regression.,value,constraints,priors
Mat52.variance,13.67876084136658,+ve,
Mat52.lengthscale,"(5,)",+ve,
Gaussian_noise.variance,0.02201300379028533,+ve,


In [14]:
print("Log Likelihood:", models['y_0']._model.log_likelihood())
models['y_0']._model

Log Likelihood: -11.531591879971002


GP_regression.,value,constraints,priors
Mat52.variance,15.55408120915987,+ve,
Mat52.lengthscale,"(5,)",+ve,
Gaussian_noise.variance,0.07359961481849882,+ve,


In [6]:
print("Log Likelihood:", models['y_0']._model.log_likelihood())
models['y_0']._model

Log Likelihood: -11.653199715971379


GP_regression.,value,constraints,priors
Mat52.variance,14.766152079938227,+ve,
Mat52.lengthscale,"(5,)",+ve,
Gaussian_noise.variance,0.07891243722611607,+ve,


In [7]:
print("Log Likelihood:", models['y_1']._model.log_likelihood())
models['y_1']._model

Log Likelihood: -8.746008594596766


GP_regression.,value,constraints,priors
Mat52.variance,13.674075435483397,+ve,
Mat52.lengthscale,"(5,)",+ve,
Gaussian_noise.variance,0.02201217750758602,+ve,


## Direct Optimization Algorithm

In [9]:
class Direct(Optimizer):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.opt_name = 'DIRECT'
        
    def opt(self, x_init, f_fp=None, f=None, fp=None):
        assert f is not None
        if self.max_iters > 6000:
            raise ValueError("Maximum iterations must be less than 6000 for DIRECT algorithm")
#         bounds = [[np.log(np.sqrt(1e-3)), np.log(np.sqrt(1e3))] for _ in range(len(x_init))]
#         bounds[-1] = [-6, ]
        bounds = [[np.sqrt(1e-3), np.sqrt(1e3)] for _ in range(len(x_init))]
        print(x_init)
        bounds[-1] = [np.exp(-6), 1]
        res = direct(f, bounds, maxT=self.max_iters)
        self.f_opt = res.fun
        self.x_opt = res.x
        self.status = 'maximum number of function evaluations exceeded'

In [16]:
inputs = [f'x_{i}' for i in range(5)]
outputs = ['y_0', 'y_1']
X = ds[inputs].standardize()
y = ds[outputs].standardize()
kern =  Matern52(input_dim =5, ARD=True)
model = GPRegression(X,y, kern)

Long story short, this really doesn't work well.

## Adam

In [8]:
class Adam(Optimizer):
    # We want the optimizer to know some things in the Optimizer implementation:
    def __init__(self, step_rate=.0002,
                                  decay=0,
                                  decay_mom1=0.1,
                                  decay_mom2=0.001,
                                  momentum=0,
                 offset=1e-8, *args, **kwargs):
        super(Adam, self).__init__(*args, **kwargs)
        self.opt_name = 'Adam (climin)'
        self.step_rate = step_rate
        self.decay = 1-1e-8
        self.decay_mom1 = decay_mom1
        self.decay_mom2 = decay_mom2
        self.momentum = momentum
        self.offset = offset

#         _check_for_climin()
        
    def opt(self, x_init, f_fp=None, f=None, fp=None):
        # We only need the gradient of the 
        assert not fp is None

        import climin

        # Do the optimization, giving previously stored parameters
        opt = climin.adam.Adam(x_init, fp,
                               step_rate=self.step_rate, decay=self.decay,
                               decay_mom1=self.decay_mom1, decay_mom2=self.decay_mom2,
                               momentum=self.momentum,offset=self.offset)

        # Get the optimized state and transform it into Paramz readable format by setting
        # values on this object:
        # Important ones are x_opt and status:
        for info in opt:
            if info['n_iter']>=self.max_iters:
                self.x_opt =  opt.wrt
                self.f_opt =  fp(opt.wrt)
                self.status = 'maximum number of function evaluations exceeded'
                break
        else: # pragma: no cover
            pass

In [9]:
inputs = [f'x_{i}' for i in range(5)]
outputs = ['y_0', 'y_1']
X = ds[inputs].standardize()
y = ds[outputs].standardize()
kern =  Matern52(input_dim =5, ARD=True)
model = GPRegression(X,y, kern)

In [13]:
model.optimize_restarts(num_restarts=10, optimizer=Adam(max_iters=1e4))

Optimization restart 1/10, f = [-1.31903979e-04 -6.29368196e-05 -3.59736149e+00 -3.19334852e+00
 -2.91457622e+00 -2.53792302e+00  4.21320339e+01]
Optimization restart 2/10, f = [-4.81777199e-01 -3.24085772e-04 -4.89054181e+00 -6.84545006e+00
 -7.28089429e+00 -4.26405445e+00  6.72103625e+01]
Optimization restart 3/10, f = [  0.80261846  -1.24877478  -9.26417142 -10.22792097  -7.39174933
  -5.46342388  61.94487183]
Optimization restart 4/10, f = [-5.50922307e-01 -5.16466282e-02 -8.14429957e+00 -7.46390096e+00
 -3.32820190e+00 -9.22968370e+00  6.33436629e+01]
Optimization restart 5/10, f = [-7.86503851e-03 -3.04180818e-01 -1.39247688e+01 -1.27230477e+01
 -1.12462712e+01 -1.35474982e+01  5.05216192e+01]
Optimization restart 6/10, f = [-3.22140178e-02 -1.44425828e-01 -7.59286167e+00 -1.57843470e+01
 -7.74360177e+00 -1.56936456e+01  5.45550640e+01]
Optimization restart 7/10, f = [-2.24454414e-03 -2.08605136e-01 -7.03362805e+00 -1.70758193e+01
 -5.16582590e+00 -2.15074836e+01  5.18178503e+01]

In [14]:
model.log_likelihood()

-114.78779767763876